In [1]:
#Importing libraries

#there are a bunch of sun-libraries that will also be imported over here instead
#of being imported randomly throughout the cooding experience
import pandas as pd # type: ignore
import numpy as np # type: ignore
import seaborn as sns # type: ignore
from matplotlib import pyplot as plt # type: ignore 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from ipywidgets import interact
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
import ipywidgets as widgets
from sklearn import linear_model
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the Iris dataset
data=pd.read_csv('diabetes.csv')

# Extract features and target
x=data.iloc[:,:-1]
y=data.iloc[:,-1] 


In [3]:
display(data.info())
display(data.describe())
display(data.size)
display(data.shape)
display(data.isna().sum())
display(data.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


None

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


6912

(768, 9)

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [4]:
def remove_outliers(data):
    df_clean = data.copy()
    initial_rows = len(df_clean)
    
    # Remove outliers for each numeric column
    for col in df_clean.select_dtypes(exclude=['object']).columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        df_clean = df_clean[(df_clean[col] >= Q1 - 1.5 * IQR) & 
                           (df_clean[col] <= Q3 + 1.5 * IQR)]
    
    # Calculate number of removed rows
    removed = initial_rows - len(df_clean)
    
    # Print removal statistics
    print(f"Initial rows: {initial_rows}")
    print(f"Rows after removal: {len(df_clean)}")
    print(f"Removed {removed} outliers ({(removed/initial_rows)*100:.2f}% of data)")
    
    return df_clean

# Apply outlier removal
data = remove_outliers(data)

Initial rows: 768
Rows after removal: 636
Removed 132 outliers (17.19% of data)


In [5]:
numeric_features = data.select_dtypes(exclude=['object']).columns
categorical_features = data.select_dtypes(include=['object']).columns

print("\nNumeric Features:", list(numeric_features))
print("Categorical Features:", list(categorical_features))


Numeric Features: ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
Categorical Features: []


In [6]:
numeric_data=data.select_dtypes(exclude=['object'])
categorical_feature=data.select_dtypes(include=['object'])

encoder=LabelEncoder()
for col in categorical_feature.columns:
    data[col]=encoder.fit_transform(categorical_feature[col])

In [7]:
def scaled_data(data):
    data_copy = data.copy()
    scaler = StandardScaler()
    data_copy[numeric_features] = scaler.fit_transform(data_copy[numeric_features]) 
    return data_copy
data_scaled = scaled_data(data)

In [8]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
def plotting(typeofplotting):
    if typeofplotting == 'Pairplot':
        # Pairplot is created for all features at once
        sns.pairplot(data, hue='Outcome')
        plt.show()
        return
    plot_features = [col for col in numeric_features if col != 'Outcome']

    n_features = len(plot_features)
    n_rows = (n_features + 1) // 2  # Ceiling division to get number of rows
    n_cols = 2

    
    # Create figure with subplots for other plot types
    plt.figure(figsize=(15, 4*n_rows))
    
    for i, col in enumerate(plot_features):
        plt.subplot(n_rows, n_cols, i+1)
        
        if typeofplotting == 'Histplot':
            sns.histplot(data=data, x=col, hue='Outcome', multiple="layer", alpha=0.5)
        
        elif typeofplotting == 'Kdeplot':
            sns.kdeplot(data=data, x=col, hue='Outcome')
        
        elif typeofplotting == 'Boxplot':
            sns.boxplot(data=data, x='Outcome', y=col)
        
        elif typeofplotting== 'Pairplot':
            sns.pairplot(data=data)

        elif typeofplotting == 'Pieplot':
            plt.pie(data[col].values,labels=data[])
        
        plt.title(f'{col} Distribution by Outcome')
        plt.xlabel(col if typeofplotting != 'Boxplot' else 'Outcome')
        plt.ylabel('Count' if typeofplotting == 'Histplot' else 'Density' if typeofplotting == 'Kdeplot' else col)
    
    plt.tight_layout()
    plt.show()

# Create interactive widget
i = interact(
    plotting,
    typeofplotting=widgets.Dropdown(
        options=['Histplot', 'Kdeplot', 'Pairplot', 'Boxplot'],
        description='Plot Type',
        value='Histplot'
    )
)

interactive(children=(Dropdown(description='Plot Type', options=('Histplot', 'Kdeplot', 'Pairplot', 'Boxplot')…